In [0]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time

print(f'PyTorch version: {torch.__version__}')

if torch.cuda.is_available()==True:
    use_cuda = True
    print(f'GPU available: {torch.cuda.get_device_name(0)} ({torch.cuda.device_count()} count)')

In [0]:
def default_device():
    if torch.cuda.is_available()==True:
        dflt_device = torch.device('cuda')
    else:
        dflt_device = torch.device('cpu')

    return dflt_device


def load_data(dir_in_str):
    E, S_pre, durations_list, min_pitch, max_pitch = torch.load(dir_in_str)
    E = E.to(device=dflt_device)
    S = []
    for tensor in S_pre:
        S.append(tensor.to(device=dflt_device))

    return E, S, durations_list, min_pitch, max_pitch


def dimensions(E,S): 
    num_event_examples, num_events , event_emb_size  = E.shape    
    num_seq_examples = len(S)
    signal_emb_size = S[0].size(1)
    
    dims = [num_event_examples, num_events , event_emb_size, num_seq_examples, signal_emb_size ]
    
    return dims
  

def prepare_data(S):
  S_pre_input = []
  first_row = torch.zeros(1,signal_emb_size).to(device=dflt_device)
  for tensor in S:
      expanded_tensor = torch.cat((first_row, tensor), dim=0)
      new_tensor = expanded_tensor[:-1, :]
      S_pre_input.append(new_tensor)
      
  conditioning_idxs_vectors = [] 
  for tensor in S:
      conditioning_indices = torch.zeros(tensor.shape[0], 1).to(device=dflt_device)
      cumulative_duration = 0
      for row in range(0, tensor.shape[0]-1):
          vector = tensor[row, :]        
          pitch_idx, rhythm_idx = list((vector != 0).nonzero())
          pitch_idx, rhythm_idx = int(pitch_idx), int(rhythm_idx)
          duration_type_idx = rhythm_idx - rhythm_idx_ini
          duration_type = durations_list[duration_type_idx]
          cumulative_duration += duration_type
          conditioning_indices[row+1] = int(cumulative_duration)
      conditioning_idxs_vectors.append(conditioning_indices)

  lengths_list = []
  for tensor in S:
      lengths_list.append(tensor.shape[0])

  S_padded = torch.nn.utils.rnn.pad_sequence(S, batch_first=True)
  S_packed = torch.nn.utils.rnn.pack_padded_sequence(S_padded, batch_first=True, lengths=lengths_list, enforce_sorted=False)

  return S_packed, S_padded, S_pre_input, lengths_list, conditioning_idxs_vectors


def create_placing_matrices(conditioning_idxs_vectors, num_events):
    placing_conditioning_matrices = []
    for vector in conditioning_idxs_vectors:
        placing_matrix = torch.zeros(vector.shape[0], num_events).to(device=dflt_device)
        for i in range(vector.shape[0]):
            placing_matrix[i, int(vector[i])] = 1
        placing_conditioning_matrices.append(placing_matrix)
        
    return placing_conditioning_matrices


def concatenate_conditioning(S_pre_input, \
                             encoded_conditioning, \
                             placing_conditioning_matrices, lengths_list):    
  S_conditioned = []
  for idx, tensor in enumerate(S_pre_input):
    placing_matrix = placing_conditioning_matrices[idx]
    dynamic_conditioning = torch.mm(placing_matrix, encoded_conditioning[idx,:,:])
    concatenated_input = torch.cat((tensor, dynamic_conditioning), dim=1)
    S_conditioned.append(concatenated_input)
  S_input = torch.nn.utils.rnn.pad_sequence(S_conditioned, batch_first=True)
  S_input = torch.nn.utils.rnn.pack_padded_sequence(S_input, batch_first=True, lengths=lengths_list, enforce_sorted=False)

  return S_input



In [0]:
class event_net(torch.nn.Module):
    
    def __init__(self, event_emb_size, event_hidden_size, event_output_size, \
                 num_event_layers, num_event_examples, num_directions):
        super(event_net, self).__init__()

        self.event_emb_size     = event_emb_size
        self.event_hidden_size  = event_hidden_size
        self.event_output_size  = event_output_size
        self.num_event_layers   = num_event_layers
        self.num_event_examples = num_event_examples
        self.num_directions     = num_directions
        
        self.event_lstm   = torch.nn.LSTM(self.event_emb_size, self.event_hidden_size, \
                                    self.num_event_layers, batch_first=True, bidirectional=True)
        self.event_linear = torch.nn.Linear(self.event_hidden_size*num_directions, self.event_output_size)
        
        self.initHidden  = self.init_hidden()
    
    def init_hidden(self):
        h_ini = (torch.zeros(self.num_event_layers*num_directions, self.num_event_examples, self.event_hidden_size),\
              torch.zeros(self.num_event_layers*num_directions, self.num_event_examples, self.event_hidden_size) )
          
    def forward(self, Events):
        event_lstm_out, event_hidden = self.event_lstm(Events, self.initHidden)
        linear_output = self.event_linear(event_lstm_out*num_event_layers)
        event_output = torch.sigmoid(linear_output)
        
        return event_output #event_lstm_out


class signal_net(torch.nn.Module):
    def __init__(self, signal_emb_size, conditioning_size, signal_hidden_size, \
                 signal_output_size, num_signal_layers, num_signal_examples):
        super(signal_net, self).__init__()
        
        self.signal_emb_size     = signal_emb_size
        self.conditioning_size   = conditioning_size
        self.signal_hidden_size  = signal_hidden_size
        self.signal_output_size  = signal_output_size
        self.num_signal_layers   = num_signal_layers
        self.num_signal_examples = num_signal_examples
        
        self.signal_lstm   = torch.nn.LSTM(self.signal_emb_size+self.conditioning_size, self.signal_hidden_size, \
                                    self.num_signal_layers, batch_first=True)
        self.signal_linear = torch.nn.Linear(self.signal_hidden_size, self.signal_output_size)
        
    def forward(self, S_input, prev_hidden):
        signal_lstm_out, signal_hidden = self.signal_lstm(S_input, prev_hidden)
        signal_linear_output = self.signal_linear(signal_lstm_out.data)
        #signal_output = torch.sigmoid(signal_linear_output)
        
        return signal_linear_output, signal_hidden



In [0]:
dflt_device = default_device()

E, S, durations_list, min_pitch, max_pitch = load_data('Parker_Dataset_unshuffled.pt')
#E, S = E[0:6,:,:], S[0:6]

num_event_examples, num_events , event_emb_size, num_signal_examples, signal_emb_size = dimensions(E,S)
rhythm_idx_ini = max_pitch - min_pitch + 1 + True

S_packed, S_padded, S_pre_input, lengths_list, conditioning_idxs_vectors = prepare_data(S)
placing_conditioning_matrices = create_placing_matrices(conditioning_idxs_vectors, num_events)

In [0]:
torch.manual_seed(12)

#Choose dimensions for event LSTM
num_event_layers = 1
event_hidden_size = 32
num_directions = 2
event_output_size = 48

#Choose dimensions for signal LSTM
num_signal_layers  = 1
signal_hidden_size = 128
signal_output_size = 89
conditioning_size  = event_output_size

#Create 1st LSTM
event_forward_pass = event_net(event_emb_size, event_hidden_size, event_output_size, \
                 num_event_layers, num_event_examples, num_directions)
event_forward_pass = event_forward_pass.to(device=dflt_device)

#Create 2nd LSTM
signal_forward_pass = signal_net(signal_emb_size, conditioning_size, signal_hidden_size, \
                 signal_output_size, num_signal_layers, num_signal_examples)
signal_forward_pass = signal_forward_pass.to(device=dflt_device)
signal_h_ini = (torch.zeros(num_signal_layers, num_signal_examples, signal_hidden_size).to(device=dflt_device),\
              torch.zeros(num_signal_layers, num_signal_examples, signal_hidden_size).to(device=dflt_device) )

weights = list(event_forward_pass.parameters()) + list(signal_forward_pass.parameters())

#Number of parameters
num_event_parameters = sum([p.numel() for p in event_forward_pass.parameters()])
print(f'Number of parameters in LSTM of events: {num_event_parameters}')

num_signal_parameters = sum([p.numel() for p in signal_forward_pass.parameters()])
print(f'Number of parameters in LSTM of signals: {num_signal_parameters}')

print(f'Total number of parameters: {num_event_parameters+num_signal_parameters}')

In [0]:
LR          = 0.05
epochs      = 3000
WeightDecay = 1e-8
Momentum    = 0.9

loss_func    = torch.nn.BCEWithLogitsLoss()
#loss_func    = torch.nn.MSELoss()
optimizer    = torch.optim.Adam(weights, lr=LR, betas=(0.9, 0.999), eps=1e-8, weight_decay = WeightDecay )
#optimizer    = torch.optim.RMSprop(weights,lr=LR, alpha=0.99, eps=1e-8, weight_decay = WeightDecay, momentum = Momentum, centered=True)
#scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=300, gamma=0.5, last_epoch=-1)
scheduler    = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0.000001, last_epoch=-1)

loss_hist  = []
for epoch in range(1, epochs+1):
  t = time.time()
  optimizer.zero_grad()
  encoded_conditioning = event_forward_pass(E)
  S_input = concatenate_conditioning(S_pre_input, encoded_conditioning, placing_conditioning_matrices, lengths_list)
  S_hat, _ = signal_forward_pass(S_input, signal_h_ini)
  Loss = loss_func(S_hat, S_packed.data)
  Loss.backward()
  optimizer.step()
  loss_hist.append(Loss.item())
  scheduler.step()
  #if epoch%200==0:
  print(f'Epoch: {epoch}, Loss: {Loss}  (Learning rate: {scheduler.get_lr()}, Time: {round(time.time()-t,4)}s')

plt.plot(loss_hist[:])
plt.xlabel('Gradient Steps')
vert_label=plt.ylabel('Loss')
vert_label.set_rotation(0)

In [0]:
hyperparameters = {'num_event_layers':num_event_layers, 'event_hidden_size':event_hidden_size,\
                       'num_directions':num_directions, 'event_output_size':event_output_size,\
                       'num_signal_layers':num_signal_layers, 'signal_hidden_size':signal_hidden_size,
                       'signal_output_size':signal_output_size, 'conditioning_size':event_output_size,\
                        'LR': LR, 'epochs':epochs, 'WeightDecay':WeightDecay, 'Momentum':Momentum }
model_parameters = [hyperparameters, event_forward_pass.state_dict(), signal_forward_pass.state_dict()]

torch.save(model_parameters, 'model_parameters.pt')
